### Индексы и булева индексация в pandas.

Вижу, что у нас остались некоторые недопонимания в этих вопросах, поэтому быстренько разберем их поподробнее.

#### 1. Индексы.

Вновь возьмем таблицу с зарплатами аналитиков:

In [2]:
import pandas as pd

df = pd.DataFrame(data={'Имя': ['Анна', 'Сергей', 'Алексей', 'Сергей', 'Екатерина'], 'Фамилия': ['Егорова', 'Тищенко', 'Маевский', 'Пеньков', 'Никонова'] ,'Январь': [1000, 1300, 800, 1100, 2000], 'Февраль': [1100, 1250, 750, 1100, 1800], 
     'Март': [950, 1320, 900, 1200, 1950]})
df

,Имя,Фамилия,Январь,Февраль,Март
0,Анна,Егорова,1000,1100,950
1,Сергей,Тищенко,1300,1250,1320
2,Алексей,Маевский,800,750,900
3,Сергей,Пеньков,1100,1100,1200
4,Екатерина,Никонова,2000,1800,1950


Обратим внимание, из чего состоит наша таблица: это данные справа и некий столбец с нумерацией слева. Столбец с нумерацией - и есть индекс таблицы.  
Главным образом индексы помогают нам получать прямой доступ к данным:

In [3]:
df.loc[1:3,'Январь']

1    1300
2     800
3    1100
Name: Январь, dtype: int64

В данном случае индекс был создан автоматически и представляет из себя последовательность натуральных чисел, однако он вовсе не обязан быть таким.

In [4]:
df.set_index([['a','b','c','d','e']])

,Имя,Фамилия,Январь,Февраль,Март
a,Анна,Егорова,1000,1100,950
b,Сергей,Тищенко,1300,1250,1320
c,Алексей,Маевский,800,750,900
d,Сергей,Пеньков,1100,1100,1200
e,Екатерина,Никонова,2000,1800,1950


Здесь мы вручную установили новые индексы, используя для этого строковые значения. 
Методу `set_index()` также можно передать столбец датафрейма:

In [5]:
df_indexed = df.set_index('Фамилия')
df_indexed

,Имя,Январь,Февраль,Март
Фамилия,,,,
Егорова,Анна,1000,1100,950
Тищенко,Сергей,1300,1250,1320
Маевский,Алексей,800,750,900
Пеньков,Сергей,1100,1100,1200
Никонова,Екатерина,2000,1800,1950


Задали в качестве индекса фамилии сотрудников. Проверим, как работает доступ к данным:

In [6]:
df_indexed.loc['Тищенко','Январь']

1300

Все сработало.  
Иногда необходимые нам данные находятся именно в индексе, поэтому нужно уметь их оттуда извлекать.

In [7]:
data_grouped = df.groupby('Имя').sum()
data_grouped

,Январь,Февраль,Март
Имя,,,
Алексей,800,750,900
Анна,1000,1100,950
Екатерина,2000,1800,1950
Сергей,2400,2350,2520


Группировка удалась на славу, но теперь поле, по которому мы группировали, ушло в индекс (при желании этого можно избежать, задав параметр `as_index=False`). Что делать, если мы хотим, например, проитерироваться по каждому имени? Поможет атрибут (какой бы вы думали?..) `index`.

In [8]:
data_grouped.index

Index(['Алексей', 'Анна', 'Екатерина', 'Сергей'], dtype='object', name='Имя')

Теперь мы легко получаем доступ к каждому имени в цикле:

In [9]:
for name in data_grouped.index:
    print('Зарплата для имени', name)
    print(data_grouped.loc[name])
    print()

Зарплата для имени Алексей
Январь     800
Февраль    750
Март       900
Name: Алексей, dtype: int64

Зарплата для имени Анна
Январь     1000
Февраль    1100
Март        950
Name: Анна, dtype: int64

Зарплата для имени Екатерина
Январь     2000
Февраль    1800
Март       1950
Name: Екатерина, dtype: int64

Зарплата для имени Сергей
Январь     2400
Февраль    2350
Март       2520
Name: Сергей, dtype: int64



#### 2. Булева индексация. 

Рассмотрим знакомую конструкцию

In [10]:
df[df['Имя']=='Сергей']

,Имя,Фамилия,Январь,Февраль,Март
1,Сергей,Тищенко,1300,1250,1320
3,Сергей,Пеньков,1100,1100,1200


Получили то, что ожидали - сотрудников с именем Сергей. Но задумывались ли вы на тем, что на самом деле мы передаем данному выражению в квадратных скобках? Самый лучший способ узнать - проверить лично:

In [14]:
df['Имя']=='Сергей'

0    False
1     True
2    False
3     True
4    False
Name: Имя, dtype: bool

Что же мы здесь имеем и как это работает? Данная конструкция вернула *булев массив* - список, состоящий из True или False. True там, где заданное условие выполнено, False там, где нет. Сравним с исходным датафреймом - действительно, Сергеи у нас в строках 1 и 3.  
Что происходит дальше, я думаю, уже понятно: эта "маска" накладывается на исходный датафрейм и возвращаются только те строки, которым соответствует значение True. 
Можем еще раз продемонстрировать, как это работает, задав булев массив явно:

In [12]:
df[[True,True,False,False,False]]

,Имя,Фамилия,Январь,Февраль,Март
0,Анна,Егорова,1000,1100,950
1,Сергей,Тищенко,1300,1250,1320


Получили те строки, для которых передали True.

Конечно же, булев массив мы получим и при более сложных логических операциях:

In [13]:
(df['Имя']=='Сергей') & (df['Январь']>1200)

0    False
1     True
2    False
3    False
4    False
dtype: bool

Как видим, по такому условию один из Сергеев у нас фильтрацию уже не прошел.